In [1]:
#Importing the necessary packages. 
import Pkg; Pkg.add("DataStructures")

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
 Resolving package versions...
  Updating `~/.julia/environments/v1.0/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.0/Manifest.toml`
 [no changes]


In [72]:
using DataStructures: DefaultDict

In [73]:
frequency_threshold = 10
training_pos_folder = "aclImdb_v1/aclImdb/train/pos"
training_neg_folder = "aclImdb_v1/aclImdb/train/neg"
testing_pos_folder = "aclImdb_v1/aclImdb/test/pos"
testing_neg_folder = "aclImdb_v1/aclImdb/test/neg"

"aclImdb_v1/aclImdb/test/neg"

In [74]:
vocabulary = DefaultDict(0)
get_vocab_word(x) = vocabulary[x] += 1

get_vocab_word (generic function with 1 method)

In [75]:
function clean_punctuation(review)
    review=replace(lowercase(review), r"<.*?>"=>" ")
    review=replace(review, r"[(?,.!;\(\)\")]"=>"")
    review
end

clean_punctuation (generic function with 1 method)

In [76]:
function load_data(folder)
    words_dict = DefaultDict(0)
    get_word(x)= words_dict[x]+=1
    num_labels = 0
    for file in readdir(folder)
        num_labels += 1
        f = open(folder*"/" * file)
        text = read(f, String)
        #println(split(clean_punctuation(lowercase(text))))
        get_word.(split(clean_punctuation(lowercase(text))))
        get_vocab_word.(split(clean_punctuation(lowercase(text))))
        close(f)
    end
    return words_dict, num_labels
end

load_data (generic function with 1 method)

In [77]:
# Loading the data.
#Training data. 
training_pos_data, pos_num = load_data(training_pos_folder)
training_neg_data, neg_num = load_data(training_neg_folder)

(DefaultDict{Any,Any,Int64}("redeemiing"=>1,"hand-smashing"=>1,"dial's"=>3,"inattentive"=>1,"f-86s"=>2,"romancethis"=>1,"sleepwalking"=>18,"photosynthesis"=>1,"henry"=>124,"young-ish"=>1…), 12500)

In [78]:
function delete_less_frequent_words(dict, freq)
    unknown_freq = 0
    for key in keys(dict)
        if dict[key] < freq
            unknown_freq += dict[key]
            delete!(dict, key)
        end
    end
end

delete_less_frequent_words (generic function with 1 method)

In [79]:
delete_less_frequent_words(vocabulary, frequency_threshold)

In [80]:
function filter_dict(dict)
    unknown_freq = 0
    for key in keys(dict)
        if !haskey(vocabulary, key)
            unknown_freq += dict[key]
            delete!(dict, key)
        end
    end
    dict["UNK"] = unknown_freq
end

fix_dict (generic function with 1 method)

In [81]:
filter_dict(training_pos_data)
filter_dict(training_neg_data)

103086

In [82]:
function calc_probs(training_pos_data, training_neg_data)
    pos_probs = Dict()
    neg_probs = Dict()
    pos_sum = sum(values(training_pos_data))
    neg_sum = sum(values(training_neg_data))
    for key in keys(training_pos_data)
        pos_probs[key] = training_pos_data[key] / pos_sum
    end
    for key in keys(training_neg_data)
        neg_probs[key] = training_neg_data[key] / neg_sum
    end
    pos_probs, neg_probs
end

calc_probs (generic function with 1 method)

In [83]:
pos_probs, neg_probs = calc_probs(training_pos_data, training_neg_data)
q_pos = pos_num / (pos_num + neg_num)
q_neg = neg_num / (pos_num + neg_num)

0.5

In [84]:
function naive_bayes_accuracy(pos_folder, neg_folder)
    pos_examples, pos_correct = test_examples(pos_folder, true)
    neg_examples, neg_correct = test_examples(neg_folder, false)
    accuracy = (pos_correct + neg_correct) / (pos_examples + neg_examples)
end

naive_bayes_accuracy (generic function with 1 method)

In [85]:
function test_examples(folder, class)
    examples_num = 0
    correct_num = 0
    for file in readdir(folder)
        examples_num += 1
        f = open(folder*"/" * file)
        text = read(f, String)
        words = split(clean_punctuation(lowercase(text)))
        pos_prob = log(q_pos) + sum([log(get(pos_probs, word, pos_probs["UNK"])) for word in words])
        neg_prob = log(q_neg) + sum([log(get(neg_probs, word, neg_probs["UNK"])) for word in words])
        if (!class && neg_prob > pos_prob) || (class && neg_prob < pos_prob)
            correct_num += 1
        end
        close(f)
    end
    examples_num, correct_num
end

test_examples (generic function with 1 method)

In [86]:
naive_bayes_accuracy(testing_pos_folder,testing_neg_folder)

0.81032